In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
#pre-process
#import packages
import os
import shutil

#rename file names
def my_removesuffix(s, suffix):
    return s[:-len(suffix)] if s.endswith(suffix) else s
def my_removeprefix(s1, prefix):
    if s1.startswith(prefix):
        return s1[len(prefix):]
    else:
        return s1
    
#reformat values (m/z, RT, MSintensity) for further analysis (data from the MVs-treated samples) 
for p in glob.iglob('IMPORTED CSV FILE'):
    basename = os.path.splitext(os.path.basename(p))[0]
    basename_without_p = basename.my_removeprefix('REDUNDANT LABEL TO BE REMOVED').my_removesuffix('REDUNDANT LABEL TO BE REMOVED')
    df = pd.read_csv(p)

    df.columns = ['mz', 'RT', 'intensity', 'no']
    from decimal import Decimal, ROUND_HALF_UP
    s_mz = df['mz']
    s_mz2= s_mz.map(lambda x: float(Decimal(str(x)).
                            quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)))
    s_mz3 = s_mz2.astype(float)
    s_RT = df['RT']
    s_RT2= s_RT.map(lambda x: float(Decimal(str(x)).
                            quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)))
    s_RT3 = s_RT2.astype(float)
    s_area = df['intensity']
    s_area2= s_area.map(lambda x: float(Decimal(str(x)).
                            quantize(Decimal('0'), rounding=ROUND_HALF_UP)))
    s_area3 = s_area2.astype(float)
    plus_df = pd.DataFrame({'mz': s_mz3, 'RT': s_RT3, 'intensity': s_area3})
    plus_df.to_csv(basename_without_p+"p.csv", index=None)
    
new_Folder = 'REFORMATED FOLDER''
for ed in glob.iglob('REFORMATED CSV FILES'): 
    shutil.move(ed,'REFORMATED FOLDER')
#"p.csv" indicates common label for files contain metabolites from the MVs-treated bacteria
#"basename_without_p+p.csv" files are defined REFORMATED CSV FILES
    
#reformat values (m/z, RT, MSintensity) for further analysis (data from non-treated samples) 
for n in glob.iglob('IMPORTED CSV FILE'):
    basename = os.path.splitext(os.path.basename(n))[0]
    basename_without_n = basename.my_removeprefix('REDUNDANT LABEL TO BE REMOVED').my_removesuffix('REDUNDANT LABEL TO BE REMOVED')
    df = pd.read_csv(n)

    df.columns = ['mz', 'RT', 'intensity', 'no']
    from decimal import Decimal, ROUND_HALF_UP
    s_mz = df['mz']
    s_mz2= s_mz.map(lambda x: float(Decimal(str(x)).
                            quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)))
    s_mz3 = s_mz2.astype(float)
    s_RT = df['RT']
    s_RT2= s_RT.map(lambda x: float(Decimal(str(x)).
                            quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)))
    s_RT3 = s_RT2.astype(float)
    s_area = df['intensity']
    s_area2= s_area.map(lambda x: float(Decimal(str(x)).
                            quantize(Decimal('0'), rounding=ROUND_HALF_UP)))
    s_area3 = s_area2.astype(float)
    minus_df = pd.DataFrame({'mz': s_mz3, 'RT': s_RT3, 'intensity': s_area3})
    minus_df.to_csv(basename_without_n+"n.csv", index=None)
    
new_Folder = 'REFORMATED FOLDER''
for ed in glob.iglob('REFORMATED CSV FILES'): 
    shutil.move(ed,'REFORMATED FOLDER')
#"n.csv" indicates common label for files contain metabolites from non-treated bacteria
#"basename_without_n+n.csv" files are defined REFORMATED CSV FILES


In [ ]:
#pre-process
#replicate reformated csv files (data from the MVs-treated samples)
for cut in glob.iglob('REFORMATED FILES in REFORMATED FOLDER'): 
    basename_p = os.path.splitext(os.path.basename(cut))[0]
    df1 = pd.read_csv(cut)
    
#correct ionization efficiency between each LC-MS batch by MS intensity of reserpine
    df1 = df1[df1['intensity'] >= 25000] #fold change of reserpine MS intensity between two LC-MS batches
    df1 = df1.reset_index()
    df1 = df1.drop('index', axis=1)
    df1.to_csv(basename_p+".csv", index=None)
new_Folder = 'CORRECTED FOLDER''
for ed in glob.iglob('CORRECTED CSV FILES'): 
    shutil.move(ed,'CORRECTED FOLDER')
    
#replicate reformated csv files (data from non-treated samples)
for cut in glob.iglob('REFORMATED CVS FILES in REFORMATED FOLDER'): 
    basename_n = os.path.splitext(os.path.basename(cut))[0]
    df2 = pd.read_csv(cut)
    
#correct ionization efficiency between each LC-MS batch by MS intensity of reserpine
    df2 = df2[df2['intensity'] >= 25000] #fold change of reserpine MSintensity between two LC-MS batches
    df2 = df2.reset_index()
    df2 = df2.drop('index', axis=1)
    df2.to_csv(basename_n+".csv", index=None)
new_Folder = 'CORRECTED FOLDER''
for ed in glob.iglob('CORRECTED CSV FILES'): 
    shutil.move(ed,'CORRECTED FOLDER')

In [ ]:
#step1
#compare masses in between the MVs-treated samples and non-treated samples
for p in glob.iglob('CORRECTED CSV FILES in CORRECTED FOLDER'):
    basename_p = os.path.splitext(os.path.basename(p))[0]
    basename_without_p = basename_p.rstrip('p')
    plus_df = pd.read_csv(p)
    for n in glob.iglob('CORRECTED CSV FILES in CORRECTED FOLDER'):
        basename_n = os.path.splitext(os.path.basename(n))[0]
        basename_without_n = basename_n.rstrip('n')
        minus_df = pd.read_csv(n)
        
        
        if basename_without_p == basename_without_n:
            list=[]
            list2=[]
            for plus_i in plus_df.itertuples():
                for minus_i in minus_df.itertuples():
                    if minus_i[1]-0.2<=plus_i[1]<=minus_i[1]+0.2 and minus_i[2]-0.19<=plus_i[2]<=minus_i[2]+0.19:
                        list.append(plus_i)
                        list2.append(minus_i)
           # 0.2 and 0.19 are used for acceptable deviation of m/z             
            overlap_with_duplicates_df = pd.DataFrame(list)
            overlap_with_duplicates_df2 = pd.DataFrame(list2)

            overlap_df = overlap_with_duplicates_df.drop_duplicates(subset=['mz', 'RT'])
            overlap_df = overlap_df.drop('Index', axis=1)
            overlap_df2 = overlap_with_duplicates_df2.drop_duplicates(subset=['mz', 'RT'])
            overlap_df2 = overlap_df2.drop('Index', axis=1)

            overlap_df.to_csv(basename_without_p+"_overlapplus.csv")
            overlap_df2.to_csv(basename_without_n+"_overlapminus.csv")

            
            #pick up masses which are present only in the MVs-treated samples
            df3 = pd.merge(plus_df,overlap_df, on=['mz', 'RT'], how='outer', indicator=True)
            plus_only_df = df3[df3['_merge'] == 'left_only'].iloc[:,:3]
            plus_only_df = plus_only_df.rename(columns={'intensity_x': 'intensity'})
            plus_only_df.to_csv(basename_without_p+"_plusonly.csv")
            #"basename_without_p+_plusonly.csv" files are defined UNIQUE MASS FILES
            
            #pick up masses which are presence in both of the MVs-treated and non-treated samples
            df4 = overlap_with_duplicates_df
            df4 = df4.drop('Index', axis=1)
            df5 = overlap_with_duplicates_df2
            df5 = df5.drop('Index', axis=1)

            df6 = pd.concat([df4, df5], axis=1, ignore_index=True)
            df6["ratio"] = np.nan
            def warizan(x):
                return x.iloc[2] / x.iloc[5]
            df6["ratio"] = df6.apply(warizan, axis=1)
            df6=df6.drop(columns=[3,4,5])
            df6.rename(columns={df6.columns[0]: 'mz'}, inplace=True)
            df6.rename(columns={df6.columns[1]: 'RT'}, inplace=True)
            df6.rename(columns={df6.columns[2]: 'intensity'}, inplace=True)
            df6.to_csv(basename_without_p+"_overlap.csv") 
            #"basename_without_p+_overlap.csv" files are defined OVERLAPPED MASS FILES  
            
new_Folder = 'UNIQUE MASS FOLDER''
for ed in glob.iglob('UNIQUE MASS FILES'): 
    shutil.move(ed,'UNIQUE MASS FOLDER')
new_Folder = 'OVERLAPPED MASS FOLDER''
for ed in glob.iglob('OVERLAPPED MASS FILES'): 
    shutil.move(ed,'OVERLAPPED MASS FOLDER')

In [ ]:
#step2
#pick up common masses between UNIQUE MASS FILEs and background masses

for p in glob.iglob('UNIQUE MASS FILES in UNIQUE MASS FOLDER'):
    basename = os.path.splitext(os.path.basename(p))[0]
    df_a = pd.read_csv(p)
    df_a = df_a.drop('Unnamed: 0', axis=1)
    df_b = pd.read_csv("BACKGROUND MASSS FILE")
    #masses from Burkholderia multivotrans and media ingredient are listed in the BACKGROUND MASSS FILE
    df_b = df_b.drop('Unnamed: 0', axis=1)
    list=[]
    list2=[]
    for df_a_i in df_a.itertuples():
        for df_b_i in df_b.itertuples():
            if df_b_i[1]-0.2<=df_a_i[1]<=df_b_i[1]+0.2 and df_b_i[2]-1.19<=df_a_i[2]<=df_b_i[2]+1.19:
                list.append(df_a_i)
                list2.append(df_b_i)

    overlap_with_duplicates_df_a = pd.DataFrame(list)
    overlap_with_duplicates_df_b = pd.DataFrame(list2)

    overlap_df = overlap_with_duplicates_df_a.drop_duplicates(subset=['mz', 'RT'])
    overlap_df.to_csv(basename+"_sameback.csv")
    #"basename+_sameback.csv" files are defined COMMON COMPOUNDS FILES

new_Folder = 'COMMON COMPOUNDS FOLDER''
for ed in glob.iglob('COMMON COMPOUNDS FILES'): 
    shutil.move(ed,'COMMON COMPOUNDS FOLDER')

In [ ]:
#step2
#exclude background masses picked up in the above from UNIQUE MASS FILEs

for p in glob.iglob('UNIQUE MASS FILES in UNIQUE MASS FOLDER'):
    basename_u = os.path.splitext(os.path.basename(p))[0]
    basename_without_u = basename_u.rstrip('_plusonly')
    df_a = pd.read_csv(p)
    df_a = df_a.drop('Unnamed: 0', axis=1)
    for n in glob.iglob('COMMON COMPOUNDS FILES in COMMON COMPOUNDS FOLDER'):
        basename_c = os.path.splitext(os.path.basename(n))[0]
        basename_without_c = basename_c.rstrip('_plusonly_sameback')
        df_b = pd.read_csv(n)
        df_b = df_b.drop('Unnamed: 0', axis=1)
        df_b = df_b.drop('Index', axis=1)
        
        if basename_without_u == basename_without_c:
            df_db = pd.merge(df_a, df_b, on=['RT', 'mz'], how='outer', indicator=True)
            plus_only_db_df = df_db[df_db['_merge'] == 'left_only'].iloc[:,:5]
            plus_only_db_df = plus_only_db_df.rename(columns={'intensity_x': 'intensity'})
            plus_only_db_df.to_csv(basename_u+"_delback.csv")
            #"basename_u+_delback.csv" files are defined UNIQUE METABOLITE FILES
            
new_Folder = 'UNIQUE METABOLITE FOLDER''
for ed in glob.iglob('UNIQUE METABOLITE FILES'): 
    shutil.move(ed,'UNIQUE METABOLITE FOLDER')

In [ ]:
#step3
#pick up the MVs-induced masses from OVERLAPPED MASS FILEs
 
for ov in glob.iglob('OVERLAPPED MASS FILES　in OVERLAPPED MASS FOLDER'):
    basename = os.path.splitext(os.path.basename(ov))[0]
    basename_without = basename.rstrip('_overlap')
    df_t = pd.read_csv(ov)
    df_t = df_t.drop('Unnamed: 0', axis=1)    
    df_t = df_t[df_t['ratio'] >= 3] # pick up >3-fold induced metabolites
    df_t = df_t.reset_index()
    df_t = df_t.drop('index', axis=1)
    df_t.to_csv(basename_without+"_ratio.csv", index=None)
    #"basename_without+_ratio.csv" files are defined >3 OVERLAPPED MASS FILES

new_Folder = '>3 OVERLAPPED MASS FOLDER''
for ed in glob.iglob('>3 OVERLAPPED MASS FILES'): 
shutil.move(ed,'>3 OVERLAPPED MASS FOLDER')

In [ ]:
#step4
#pick up common masses between >3 OVERLAPPED MASS FILEs and background masses
   
for p in glob.iglob('>3 OVERLAPPED MASS FILES in >3 OVERLAPPED MASS FOLDER'):
    basename = os.path.splitext(os.path.basename(p))[0]
    df_a = pd.read_csv(p)
    df_b = pd.read_csv("BACKGROUND COMPOUNDS CSV FILE")
    #masses from Burkholderia multivotrans and media ingredient are listed in the BACKGROUND MASSS FILE
    df_b = df_b.drop('Unnamed: 0', axis=1)
    list=[]
    list2=[]
    for df_a_i in df_a.itertuples():
        for df_b_i in df_b.itertuples():
            if df_b_i[1]-0.2<=df_a_i[1]<=df_b_i[1]+0.2 and df_b_i[2]-1.19<=df_a_i[2]<=df_b_i[2]+1.19:
                list.append(df_a_i)
                list2.append(df_b_i)

    overlap_with_duplicates_df_a = pd.DataFrame(list)
    overlap_with_duplicates_df_b = pd.DataFrame(list2)

    overlap_df = overlap_with_duplicates_df_a.drop_duplicates(subset=['mz', 'RT'])
    overlap_df.to_csv(basename+"_sameback.csv")
    #"basename+_sameback.csv" files are defined COMMON OVERLAPPED COMPOUNDS FILES
    
new_Folder = 'COMMON OVERLAPPED COMPOUNDS FILES FOLDER''
for ed in glob.iglob('COMMON OVERLAPPED COMPOUNDS FILES'): 
shutil.move(ed,'COMMON OVERLAPPED COMPOUNDS FOLDER')

In [ ]:
#step4
#exclude background masses from >3 OVERLAPPED MASS FILEs

for p in glob.iglob('>3 OVERLAPPED MASS CSV FILES in >3 OVERLAPPED MASS FOLDER'):
    basename_o = os.path.splitext(os.path.basename(p))[0]
    basename_without_o = basename_o.rstrip('_ratio')
    df_a = pd.read_csv(p)
    for n in glob.iglob('COMMON OVERLAPPED COMPOUNDS FILES in COMMON OVERLAPPED COMPOUNDS FOLDER'):
        basename_c = os.path.splitext(os.path.basename(n))[0]
        basename_without_c = basename_c.rstrip('_ratio_sameback')
        df_b = pd.read_csv(n)
        df_b = df_b.drop('Unnamed: 0', axis=1)
        df_b = df_b.drop('Index', axis=1)
        
        if basename_without_o == basename_without_c:
            df_db = pd.merge(df_a, df_b, on=['RT', 'mz'], how='outer', indicator=True)
            plus_only_db_df = df_db[df_db['_merge'] == 'left_only'].iloc[:,:4]
            plus_only_db_df = plus_only_db_df.rename(columns={'intensity_x': 'intensity'})
            plus_only_db_df.to_csv(basename_o+"_delback.csv")
        #"basename_o+_delback.csv" files are defined INDUCED METABOLITE FILES
        
new_Folder = 'INDUCED METABOLITE FOLDER''
for ed in glob.iglob('INDUCED METABOLITE FILES'): 
shutil.move(ed,'INDUCED METABOLITE FOLDER')